In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [7]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()
df = market.to_df()
df.head()

,maxLeverage,name,onlyIsolated,szDecimals,dayNtlVlm,funding,impactPxs,markPx,midPx,openInterest,oraclePx,premium,prevDayPx
0,50,BTC,False,5,6.708418e+08,-0.000003,"[61093.0, 61098.2]",61097.00000,61093.50000,1.550142e+03,61132.00000,-0.000553,63896.0000
1,50,ETH,False,4,4.781993e+08,-0.000005,"[2976.8, 2977.16]",2977.20000,2976.85000,3.535299e+04,2979.00000,-0.000618,3091.4000
2,20,ATOM,False,2,3.885378e+06,0.000007,"[8.0762, 8.0772]",8.07670,8.07710,9.371892e+04,8.08080,-0.000445,8.0916
3,20,MATIC,False,1,9.087017e+06,0.000013,"[0.66823, 0.66851]",0.66828,0.66834,2.215322e+06,0.66845,0.000000,0.6991
4,20,DYDX,False,1,2.517911e+06,0.000013,"[2.11922, 2.12032]",2.11980,2.11975,7.949052e+05,2.12000,0.000000,2.1092


In [8]:
# get top n coins

def get_top_n_coins(n=60):
    top_n_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(n)
    top_n_coins_df = top_n_coins_df.reset_index(drop=True)
    top_n_coins_df.head()

    # get first coin name
    top_n_coins = top_n_coins_df.get('name')
    first_coin = top_n_coins[0]
    print("top 1st coin:", first_coin)
    return top_n_coins

top_n_coins = get_top_n_coins(5)

top 1st coin: BTC


In [9]:
%time
import time
from hyperliquid.info import Info
from hyperliquid.utils import constants as hl_constants
import utils.now_timestamp as now_timestamp 
import models.candles as candles
import importlib
importlib.reload(candles)
importlib.reload(now_timestamp)
info = Info(hl_constants.MAINNET_API_URL, skip_ws=True)

def fetch_coins_by_names(names):
    start = now_timestamp.get_timestamp_from_today_midnight_utc(days=-30)
    end = now_timestamp.get_timestamp_from_today_midnight_utc(days=0)
    error_coins = []
    for coin in names:
        try:
            time.sleep(0.2)
            print(coin)
            result = info.candles_snapshot(coin, '15m', start, end)
            _candles = candles.CandleList.from_response(result)
            _candles.to_csv()
        except Exception as e:
            print(f"coin {coin}: not fetched")
            print(e)
            error_coins.append(coin)
            continue

fetch_coins_by_names(top_n_coins)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.87 µs
BTC
ETH
SOL
TIA
SUI
